In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category = UserWarning)

In [10]:
'''
Loading the data
'''

folder_path = "/kaggle/input/store-sales-time-series-forecasting/"

holiday_event_df = pd.read_csv(os.path.join(folder_path,"holidays_events.csv"),
                               dtype={'type': 'category',
                                      'locale': 'category',
                                      'locale_name': 'category',
                                      'description': 'category',
                                      'transferred': 'bool'},
                               parse_dates=['date'],
                               infer_datetime_format=True)

holiday_event_df = holiday_event_df.set_index('date').to_period('D')

holiday_event_df_duplicates = holiday_event_df[holiday_event_df.index.duplicated(keep=False)] #Find duplicate values in holidays

holiday_event_df_without_duplicates = holiday_event_df[~holiday_event_df.index.duplicated(keep='first')] #Handle duplicate values in holidays

oil_df = pd.read_csv(os.path.join(folder_path,"oil.csv"),
                     parse_dates=['date'],
                     infer_datetime_format=True)

oil_df = oil_df.set_index('date').to_period('D')

oil_df = oil_df.interpolate() #Handle missing values in oil prices

oil_df.iloc[0] = oil_df.iloc[1] #Handle missing values in oil prices

oil_df.rename(columns={"dcoilwtico": "oil_price"}, inplace = True)

stores_df = pd.read_csv(os.path.join(folder_path,"stores.csv"))

transaction_df = pd.read_csv(os.path.join(folder_path,"transactions.csv"),
                             parse_dates=['date'],
                             infer_datetime_format=True)

transaction_df['date'] = transaction_df['date'].dt.to_period('D')
transaction_df = transaction_df.set_index(['date', 'store_nbr']).sort_index()

train_df = pd.read_csv(os.path.join(folder_path,"train.csv"),
                                        usecols=['store_nbr', 'family', 'date','sales'],
                                        dtype={'store_nbr': 'category',
                                               'family': 'category',
                                               'sales': 'float'},
                                        parse_dates=['date'],
                                        infer_datetime_format=True)

train_df['date'] = train_df.date.dt.to_period('D')
train_df = train_df.set_index(['store_nbr', 'family','date']).sort_index()

competition_test_df = pd.read_csv(os.path.join(folder_path,"test.csv"),
                                  usecols=['id','store_nbr', 'family', 'date', 'onpromotion'],
                                  dtype={'store_nbr': 'category',
                                         'family': 'category',
                                         'onpromotion': 'uint32'},
                                  parse_dates=['date'],
                                  infer_datetime_format=True)

competition_test_df['date'] = competition_test_df.date.dt.to_period('D')
competition_test_df = competition_test_df.set_index(['store_nbr', 'family', 'date']).sort_index()


In [11]:
'''
Building a multivariate linear regression model to predict sales by store and family.
'''

y = train_df.unstack(level = ['store_nbr', 'family']).loc['2016':'2017'].sort_index()

X = y.copy()

fourier = CalendarFourier(freq = 'M', order = 4)
dp = DeterministicProcess(index=y.index,
                          constant=True,
                          order=1,
                          seasonal = True,
                          additional_terms=[fourier],
                          drop=True)

X = dp.in_sample()

X['NewYear'] = (X.index.dayofyear == 1)
X['holiday'] = X.index.to_series().isin(holiday_event_df.index)

y_train, y_test = y[:"2017-06-01"], y["2017-06-02":]
X_train, X_test = X.loc[:"2017-06-01"], X.loc["2017-06-02":]

model = LinearRegression().fit(X_train, y_train)

y_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y_test.columns)

print("\nTest R2 of the multivariate linear regression model: {}\n".format(r2_score(y_test.stack(level = ['store_nbr', 'family']), 
                                                                                    y_pred.stack(level = ['store_nbr', 'family']))))



Test R2 of the multivariate linear regression model: 0.9086676734679275



In [12]:
'''
Predicting sales for the competition dataset
'''

X_competition = dp.out_of_sample(steps=16)
X_competition.index.name = 'date'
X_competition['NewYear'] = (X_competition.index.dayofyear == 1)
X_competition['holiday'] = X_competition.index.to_series().isin(holiday_event_df.index)

y_submit = pd.DataFrame(model.predict(X_competition), index=X_competition.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(competition_test_df.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)

print("\nBelow are the predictions for the competition data:")
print(y_submit)


Below are the predictions for the competition data:
                                                      id        sales
date       store_nbr family                                          
2017-08-16 1         AUTOMOTIVE                  3000888     3.984071
                     BABY CARE                   3000889     0.000000
                     BEAUTY                      3000890     3.784154
                     BEVERAGES                   3000891  2442.775081
                     BOOKS                       3000892     1.130461
...                                                  ...          ...
2017-08-31 9         POULTRY                     3029395   378.581017
                     PREPARED FOODS              3029396   130.861832
                     PRODUCE                     3029397  1486.169327
                     SCHOOL AND OFFICE SUPPLIES  3029398     2.875903
                     SEAFOOD                     3029399    14.300924

[28512 rows x 2 columns]
